In [21]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [22]:
import tensorflow as tf
import seaborn as sn
import numpy as np
import pandas as pd
import math
import datetime
import platform
import os

In [23]:
IMAGE_DIRECTORY = "../Datasets/HandWritten-Nist/dataset"

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_DIRECTORY,
    labels='inferred',
    label_mode='int',
    class_names=[str(i) for i in range(10)],
    color_mode='grayscale',
    image_size=(128, 128),
    shuffle=True,
    seed=123,
    interpolation='bilinear',
    follow_links=False,
    data_format=None,
    verbose=True
)
# train_ds, test_ds = tf.keras.utils.split_dataset(dataset, left_size=0.8)
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     IMAGE_DIRECTORY,
#     labels='inferred',
#     label_mode='int',
#     class_names=[str(i) for i in range(10)],
#     color_mode='grayscale',
#     image_size=(128, 128),
#     shuffle=True,
#     seed=123,
#     validation_split=0.8,
#     subset="training",
#     interpolation='bilinear',
#     follow_links=False,
#     data_format=None,
#     verbose=False
# )
# test_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     IMAGE_DIRECTORY,
#     labels='inferred',
#     label_mode='int',
#     class_names=[str(i) for i in range(10)],
#     color_mode='grayscale',
#     image_size=(128, 128),
#     shuffle=True,
#     seed=123,
#     validation_split=0.2,
#     subset="validation",
#     interpolation='bilinear',
#     follow_links=False,
#     data_format=None,
#     verbose=False
# )

Found 100000 files belonging to 10 classes.


In [24]:
# Configuring the dataset for performance
# Enable caching and prefetching (overlap preprocessing and model execution while training)
AUTOTUNE = tf.data.AUTOTUNE

# Set up a pre-processing function. It is just going to be normalizing here, but
# that hopefully should save time later?
# def preprocess_image(image, label):
# 	image = image / 255.0
# 	return image, label

# # Cache to disk instead of memory because we got some large datasets
# dataset = dataset.cache('/tmp/dataset_cache')

# # Apply preprocessing
# # dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# # Prefetching..? Gotta be honest, this seems to help but idk how considering all
# # the images are local
# dataset = dataset.prefetch(buffer_size=2)

# train_ds = dataset.shuffle(1000)

# train_ds = dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
train_ds = dataset.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [25]:
print(dataset.class_names)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [26]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Standardize
# Bring it down from 0..255 to [0,1]


In [27]:
model = tf.keras.models.Sequential()
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
IMAGE_CHANNELS = 1

model.add(tf.keras.layers.Rescaling(
	scale=1./255,
    input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
))

model.add(tf.keras.layers.Convolution2D(
    # input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
    kernel_size=5,
    filters=8,
    strides=1,
    activation=tf.keras.activations.relu,
    kernel_initializer=tf.keras.initializers.VarianceScaling(),
))

model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

model.add(tf.keras.layers.Convolution2D(
    kernel_size=5,
    filters=16,
    strides=1,
    activation=tf.keras.activations.relu,
    kernel_initializer=tf.keras.initializers.VarianceScaling()
))

model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(
    units=128,
    activation=tf.keras.activations.relu
))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(
    units=10,
    activation=tf.keras.activations.softmax,
    kernel_initializer=tf.keras.initializers.VarianceScaling()
))

In [28]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)         │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 124, 124, 8)    │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 58, 58, 16)     │         3,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 29, 29, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 13456)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     1,722,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,727,210 (6.59 MB)

 Trainable params: 1,727,210 (6.59 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Compile the model
# Uses Adam
# Idk what the fuck adam stands for. Look it up
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=adam_optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy ,
    # loss=tf.keras.losses.CTC,
    metrics=['accuracy']
)

In [30]:
log_dir=".logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

training_history = model.fit(
    train_ds,
    # y_train_re,
    epochs=10,
    # validation_data=(x_validation_normalized, y_validation_re),
    callbacks=[tensorboard_callback]
)

print("The model has successfully trained")

Epoch 1/10


/home/nuttynoah/Downloads/Handwriting-Recognition-CS584/.venv/lib/python3.10/site-packages/keras/src/callbacks/tensorboard.py:667: UserWarning: Model failed to serialize as JSON. Ignoring... SparseCategoricalCrossentropy.get_config() missing 1 required positional argument: 'self'
  warnings.warn(f"Model failed to serialize as JSON. Ignoring... {exc}")
I0000 00:00:1729705536.439087  252666 service.cc:146] XLA service 0x7efbbc0046a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729705536.439146  252666 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6


   1/3125 ━━━━━━━━━━━━━━━━━━━━ 4:51:51 6s/step - accuracy: 0.0938 - loss: 2.3726

I0000 00:00:1729705539.360843  252666 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 221s 69ms/step - accuracy: 0.7357 - loss: 0.8475
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9549 - loss: 0.1575
Epoch 3/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9738 - loss: 0.0913
Epoch 4/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.9812 - loss: 0.0664
Epoch 5/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9848 - loss: 0.0529
Epoch 6/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 23s 7ms/step - accuracy: 0.9869 - loss: 0.0454
Epoch 7/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9898 - loss: 0.0367
Epoch 8/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9911 - loss: 0.0327
Epoch 9/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.9916 - loss: 0.0285
Epoch 10/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.9930 - loss: 0.0245
The model has successfully trained
